In [3]:
import json
import pyhf
pyhf.set_backend('jax')


import sys
sys.path.insert(1, '/Users/malinhorstmann/Documents/pyhf_pymc/src')
from pyhf_pymc import MH_inference
from pyhf_pymc import HMC_inference

# Model

In [4]:
### Simple pyhf model
dummy_model = pyhf.Model(
    {'channels': [{'name': 'singlechannel',
    'samples': [
    {'name': 'signal',
     'data': [6, 6, 3],
     'modifiers': [
         {'name': 'mu', 'type': 'normfactor', 'data': None}]},

    {'name': 'background',
     'data': [55, 55, 55],''
     'modifiers': [
        ## Staterror / Normal
        {"name": "my_staterror","type": "staterror","data": [2.0, 2.0, 2.4],},
        ## Lumi / Normal
        {'name': 'lumi', 'type': 'lumi', 'data': None},
        ## Correlated / Normal
        {'name': 'corr_bkg', 'type': 'histosys','data': {'hi_data': [65, 56, 67], 'lo_data': [40, 40, 43]}},
        {'name': 'corr_bkg1', 'type': 'histosys','data': {'hi_data': [65, 65, 66], 'lo_data': [40, 40, 40]}},
        {'name': 'corr_bkg2', 'type': 'histosys','data': {'hi_data': [66, 65, 60], 'lo_data': [40, 40, 39]}},
        ## Uncorrelated / Poisson
        {'name': 'uncorr_bkg', 'type': 'shapesys','data': [7, 8, 7.17]},
        {'name': 'uncorr_bkg1', 'type': 'shapesys','data': [7, 8, 6.7]},
        {'name': 'uncorr_bkg2', 'type': 'shapesys','data': [7.27, 9, 7]},
        
         ]},    
                                 
    ]},
    ],
    "parameters": [
            {
                "name": "lumi",
                "auxdata": [1.0],
                "sigmas": [0.017],
                "bounds": [[0.915, 1.085]],
                "inits": [1.0],
            }
        ],}
)

dummy_nBins = len(dummy_model.expected_actualdata(dummy_model.config.suggested_init()))

### Observations
dummy_obs = dummy_model.expected_actualdata(dummy_model.config.suggested_init())

In [5]:
with open('SRee_SRmm_Srem.json') as serialized:
    spec = json.load(serialized)

workspace = pyhf.Workspace(spec)
DL_model = workspace.model()
DL_obs = workspace.data(DL_model, include_auxdata=False)
DL_nBins = len(DL_model.expected_actualdata(DL_model.config.suggested_init()))


with open('ttbar_ljets_xsec_inclusive_pruned.json') as serialized:
    spec = json.load(serialized)

workspace = pyhf.Workspace(spec)
ttbar_model = workspace.model()
ttbar_obs = workspace.data(ttbar_model, include_auxdata=False)
ttbar_nBins = len(DL_model.expected_actualdata(ttbar_model.config.suggested_init()))

FileNotFoundError: [Errno 2] No such file or directory: 'SRee_SRmm_Srem.json'